In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [ ]:
# Global settings
analysis_start = '2007-12-31'
analysis_end = '2021-01-31'

output_filename = 'f_capW-quintiles' + '.xlsx'
esg_fixed = False
equal_mkt_weights = False
method_split = 'tiles'  # 'mean' 'median' 'tiles'

In [ ]:
FF_link = ''
ff = pd.read_excel(FF_link).dropna(how='all')
ff = ff.set_index('date', drop=True)
ff = ff / 100

ff = ff[['Mkt-RF', 'SMB', 'HML', 'MOM', 'RF']]

ff.tail()

In [ ]:
# load all data from master Google Sheets file
ESG_link = ''

esg_meta = pd.read_excel(ESG_link, sheet_name='MetaData').dropna(how='all')
esg_cap = pd.read_excel(ESG_link, sheet_name='FreeFloat Mkt. Cap. CM').dropna(how='all')
esg_returns = pd.read_excel(ESG_link, sheet_name='1-CM Total Return (incl. div)').dropna(how='all')
esg_snp = pd.read_excel(ESG_link, sheet_name='SPX Historical Constituents CY').dropna(how='all')
esg_social = pd.read_excel(ESG_link, sheet_name='Re_Soc FY Absolute').dropna(how='all')
esg_governance = pd.read_excel(ESG_link, sheet_name='Re_Gov FY Absolute').dropna(how='all')
esg_environment = pd.read_excel(ESG_link, sheet_name='Re_Env FY Absolute').dropna(how='all')

In [ ]:
# this cell parses RIC (ticker.exchange) to (ticker) - fx. "GOOG.OQ" -> "GOOG"
def parse_ric(ric):
    if ric == 'Date':
        return ric
    
    ric = re.findall(string=ric, pattern='([A-Za-z]+)\..+')[0]

    return ric


# returns and cap
esg_cap.columns = list(map(parse_ric, esg_cap.columns))
esg_returns.columns = list(map(parse_ric, esg_returns.columns))

# ESG metrics
esg_social.columns = list(map(parse_ric, esg_social.columns))
esg_governance.columns = list(map(parse_ric, esg_governance.columns))
esg_environment.columns = list(map(parse_ric, esg_environment.columns))

In [ ]:
def get_snp_tickers(snp, period):
    """Gets vectors of tickers at `period` year-end

    Parameters:
    snp (DataFrame): DataFrame of SNP constituents
    period ('YYYY-MM-DD' formatted string): year-end 

    Returns:
    list: List of SNP constituents at specified year-end

    """
    # some tickers have halted trading
    halted = ['TIF', 'CXO']

    # get tickers
    snp = snp.loc[snp['Date'] == period]
    snp = snp.dropna(how='any', axis=1)
    snp = snp.drop(columns='Date')
    snp = snp.values[0]

    # clean tickers
    snp = list(set(snp) - set(halted))  # tickers not equal to TIF or CXO
    snp.append('Date')

    return snp

snp_tickers = get_snp_tickers(esg_snp, '2020-12-31')

In [ ]:
def clean_esg(df):
    # memory safety:
    df = df.copy()

    df = df[snp_tickers]
    df = df.set_index('Date')
    df.index.name = None
    df = df.loc[df.index >= analysis_start]
    df.index = df.index.map(lambda x: x + pd.tseries.offsets.MonthEnd(0))

    return df

# social
soc = clean_esg(esg_social)

# environment
env = clean_esg(esg_environment)

# governance
gov = clean_esg(esg_governance)

# returns
returns = clean_esg(esg_returns)

# market cap
cap = clean_esg(esg_cap)

In [ ]:
# set ESG-rebalance at june following year (FY2019 -> 30-06-2020)
gov.index = gov.index.map(lambda x: x + relativedelta(months=+6))
soc.index = soc.index.map(lambda x: x + relativedelta(months=+6))
env.index = env.index.map(lambda x: x + relativedelta(months=+6))

In [ ]:
print('Missing for Social:')
display(soc.columns[soc.isna().all()])
print('Missing for Environment:')
display(env.columns[env.isna().all()])
print('Missing for Governance:')
display(gov.columns[gov.isna().all()])

# drop all NaN values in columns (where all are NaN)
soc = soc.dropna(how='all', axis=1)
env = env.dropna(how='all', axis=1)
gov = gov.dropna(how='all', axis=1)

In [ ]:
def esg_merge(esg, slice_method, fixed=True, lag=0, q1=0.2, q2=0.8):
    # memory safety:
    esg = esg.copy()

    # we forward fill the ESG scores
    esg = esg.ffill()

    # ability to lag ESG scores
    if lag > 0:
        esg = esg.shift(lag)

    esg = esg.transpose()

    # different splitting methods
    methods = {
        # IDEA: implement top 10% vs bottom 10%
        'mean': {
            'metric1': np.nanmean,
            'metric2': np.nanmean,
            'interval': [False, True]
        },
        'median': {
            'metric1': np.nanmedian,
            'metric2': np.nanmedian,
            'interval': [False, True]
        },
        'tiles': {
            'metric1': np.nanquantile,
            'metric2': np.nanquantile,
            'interval': [False, np.inf, True]
        }
    }
    
    # esg data
    for col in esg.columns:
        # here we need to implement pd.cut(...) method instead 

        if slice_method == 'tiles':
            method = methods[slice_method]

            esg[col] = pd.cut(x=esg[col],
                              bins=[-1, method['metric1'](esg[col], q=q1), method['metric2'](esg[col], q=q2), 101],
                              labels=method['interval'],
                              duplicates='drop')
        
            # safety replace
            #esg[col] = esg[col].replace('NaN', np.nan)
            
        else:
            method = methods[slice_method]
            esg[col] = pd.cut(x=esg[col], 
                              bins=[-1, method['metric1'](esg[col]), method['metric2'](esg[col]), 101], 
                              labels=method['interval'],
                              duplicates='drop')
            # safety replace
            #esg[col] = esg[col].replace('NaN', np.nan)


    if fixed:
        for col in esg.columns:
            esg[col] = esg[esg.columns.max()]

    esg = esg.reindex(sorted(esg.columns, reverse=True), axis=1)

    return esg


soc_screen = esg_merge(esg=soc, slice_method=method_split, fixed=esg_fixed)  # merge and split ESG's
env_screen = esg_merge(esg=env, slice_method=method_split, fixed=esg_fixed)  # merge and split ESG's
gov_screen = esg_merge(esg=gov, slice_method=method_split, fixed=esg_fixed)  # merge and split ESG's

In [ ]:
def expand_y2m(y_df, start=analysis_start, end=analysis_end):
    # memory safety
    y_df = y_df.copy()

    # create monthly index and df
    idx = pd.date_range(start=start, end=end, freq='M')
    m_df = pd.DataFrame(index=idx)

    y_df = m_df.join(y_df.transpose()).ffill()
    y_df = y_df.replace(np.inf, np.nan)

    return y_df


soc_screen = expand_y2m(soc_screen)  # expand screening bool df from YY to MM
env_screen = expand_y2m(env_screen)  # expand screening bool df from YY to MM
gov_screen = expand_y2m(gov_screen)  # expand screening bool df from YY to MM

In [ ]:
def portfolio_weights(esg, mkt_cap, equal_w):
    # memory safety:
    esg, mkt_cap = esg.copy(), mkt_cap.copy()

    # transpose data
    esg = esg.transpose()
    mkt_cap = mkt_cap.transpose()

    def weights(cap_vector):
        #print(np.sum(cap_vector), cap_vector.name)
        w = cap_vector.apply(lambda x: x / np.sum(cap_vector))

        return w
    
    for col in esg.columns:
        high_esg = esg[col].dropna().astype('bool')
        low_esg = high_esg.loc[~high_esg]
        high_esg = high_esg.loc[high_esg]
        
        if equal_w:
            mkt_cap[col].loc[~mkt_cap.index.isin(pd.concat([high_esg, low_esg]).index)] = np.nan

            mkt_cap.loc[high_esg.index, col] = mkt_cap.loc[high_esg.index, col].notnull().astype('int')
            mkt_cap.loc[low_esg.index, col] = mkt_cap.loc[low_esg.index, col].notnull().astype('int')

            mkt_cap[col] = mkt_cap[col].replace(0, np.nan)

            # relative weight to number of non NaN observations
            num_high = mkt_cap.loc[high_esg.index][col].dropna().shape[0]
            num_low = mkt_cap.loc[low_esg.index][col].dropna().shape[0]
            
            if num_high == 0:
                eq_w_high = 0
            else:
                eq_w_high = 1 / num_high
            
            if num_low == 0:
                eq_w_low = 0
            else:
                eq_w_low = 1 / num_low
            
            mkt_cap.loc[high_esg.index, col] = mkt_cap.loc[high_esg.index][col].replace(1, eq_w_high)
            mkt_cap.loc[low_esg.index, col] = mkt_cap.loc[low_esg.index][col].replace(1, eq_w_low)

        else:
            mkt_cap[col].loc[~mkt_cap.index.isin(pd.concat([high_esg, low_esg]).index)] = np.nan

            mkt_cap.loc[high_esg.index, col] = weights(mkt_cap.loc[high_esg.index, col])
            mkt_cap.loc[low_esg.index, col] = weights(mkt_cap.loc[low_esg.index, col])

    return esg, mkt_cap



soc_screen, w_soc = portfolio_weights(esg=soc_screen, mkt_cap=cap, equal_w=equal_mkt_weights)  # calc weights
soc_screen, w_soc = soc_screen.dropna(how='all', axis=1), w_soc.dropna(how='all', axis=1)

env_screen, w_env = portfolio_weights(esg=env_screen, mkt_cap=cap, equal_w=equal_mkt_weights)  # calc weights
env_screen, w_env = env_screen.dropna(how='all', axis=1), w_env.dropna(how='all', axis=1)

gov_screen, w_gov = portfolio_weights(esg=gov_screen, mkt_cap=cap, equal_w=equal_mkt_weights)  # calc weights
gov_screen, w_gov = gov_screen.dropna(how='all', axis=1), w_gov.dropna(how='all', axis=1)

In [ ]:
def sort_dataframes(returns_df, weights_df, screen_df):
    # memory safety:
    returns_df, weights_df, screen_df = returns_df.copy(), weights_df.copy(), screen_df.copy()

    # transpose
    weights_df, screen_df = weights_df.transpose(), screen_df.transpose()

    # sorting - rows
    returns_df, weights_df, screen_df = returns_df.sort_index(), weights_df.sort_index(), screen_df.sort_index()

    # drop excess return observations
    returns_df = returns_df[returns_df.index.isin(screen_df.index)]

    # drop stocks with missing information
    returns_df = returns_df[screen_df.columns]
    weights_df = weights_df[screen_df.columns]
    

    # sorting - columns
    def sort_cols(df):
        # memory safety:
        df = df.copy()

        df = df.reindex(sorted(df.columns), axis=1)

        return df

    returns_df, weights_df, screen_df = sort_cols(returns_df), sort_cols(weights_df), sort_cols(screen_df)

    return returns_df, weights_df, screen_df


def get_reduced_matrix(returns_df, weights_df, screen_df, high_low):
    '''
    This is clever - we cast the screening dataframe as a integer (dropping all False)

    Then we can multiply returns and weights due to synchronous sorting and indexing
    '''
    df = screen_df.copy()

    if high_low == 'High':
        df = df.astype('float').astype('Int64')
        df = df.replace(0, np.nan)

        return (df * returns_df * weights_df)

    if high_low == 'Low':
        df = df.astype('float').astype('Int64')
        df = df.replace(1, np.nan)
        df = df.replace(0, 1)

        return (df * returns_df * weights_df)


returns, w_soc, soc_screen = sort_dataframes(returns, w_soc, soc_screen)
low_soc = get_reduced_matrix(returns, w_soc, soc_screen, 'Low')
high_soc = get_reduced_matrix(returns, w_soc, soc_screen, 'High')

returns, w_env, env_screen = sort_dataframes(returns, w_env, env_screen)
low_env = get_reduced_matrix(returns, w_env, env_screen, 'Low')
high_env = get_reduced_matrix(returns, w_env, env_screen, 'High')

returns, w_gov, gov_screen = sort_dataframes(returns, w_gov, gov_screen)
low_gov = get_reduced_matrix(returns, w_gov, gov_screen, 'Low')
high_gov = get_reduced_matrix(returns, w_gov, gov_screen, 'High')

In [ ]:
factors = pd.DataFrame(index=high_soc.index)

factors['SUS_soc'] = (high_soc.sum(axis=1) - low_soc.sum(axis=1))
factors['SUS_env'] = (high_env.sum(axis=1) - low_env.sum(axis=1))
factors['SUS_gov'] = (high_gov.sum(axis=1) - low_gov.sum(axis=1))

factors = factors.join(ff)

factors.to_excel(output_filename)

factors